# AWS Bedrock Agents with LangGraph & LangSmith
## Production-Grade Agent Orchestration and Observability Lab

**Author:** Senior AWS Solutions Architect & GenAI Specialist  
**Duration:** 3-4 hours  
**Level:** Advanced  
**Last Updated:** February 2026

---

## 🎯 Lab Overview

This comprehensive hands-on lab demonstrates enterprise-grade AI agent development using:

- **AWS Bedrock Agents** - Managed agent framework with custom action groups
- **LangGraph** - Advanced agent orchestration with state management
- **LangSmith** - Full observability, tracing, and debugging
- **Multiple Bedrock Models** - Claude 3 Haiku, Claude 3 Sonnet, Amazon Titan

### 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                         User Interface                       │
└────────────────────────┬────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────────┐
│              LangGraph Agent Orchestrator                    │
│          (State Management + Routing Logic)                  │
└──────┬─────────────────────────────┬────────────────────────┘
       │                             │
       ▼                             ▼
┌──────────────┐              ┌──────────────┐
│   Bedrock    │              │   Bedrock    │
│   Agent 1    │              │   Agent 2    │
│  (Weather)   │              │  (Booking)   │
└──────┬───────┘              └──────┬───────┘
       │                             │
       │   Action Groups             │
       ▼                             ▼
┌──────────────┐              ┌──────────────┐
│   Lambda     │              │   Lambda     │
│   Function   │              │   Function   │
│  (Weather)   │              │  (Booking)   │
└──────────────┘              └──────────────┘
       │                             │
       └─────────────┬───────────────┘
                     ▼
          ┌─────────────────────┐
          │   LangSmith Trace   │
          │   & Observability   │
          └─────────────────────┘
```

### 🎓 Learning Objectives

By completing this lab, you will:

1. ✅ Create AWS Bedrock Agents with custom action groups
2. ✅ Design OpenAPI 3.0 schemas for agent capabilities
3. ✅ Implement Lambda functions as action group backends
4. ✅ Build multi-step agent workflows with LangGraph
5. ✅ Implement stateful conversations with memory
6. ✅ Add human-in-the-loop approval patterns
7. ✅ Handle errors gracefully with fallback strategies
8. ✅ Trace and debug agents with LangSmith
9. ✅ Implement single-turn and multi-turn interaction patterns
10. ✅ Deploy production-ready agent systems

### ⚠️ Prerequisites

Before starting, ensure you have:

- ✅ AWS Account with Bedrock access
- ✅ SageMaker Studio or SageMaker AI Studio environment
- ✅ IAM permissions for: Bedrock, Lambda, IAM, CloudWatch
- ✅ Enabled Bedrock models (we'll verify this below)
- ✅ LangSmith account (optional - for observability)

---

## Part 1: Environment Setup and Dependency Installation

Let's start by installing all required dependencies and verifying our AWS environment.

In [ ]:
# Install required packages
# Note: Run this cell first and restart kernel if prompted

!pip install -q --upgrade pip
!pip install -q boto3>=1.34.0
!pip install -q langchain>=0.1.0
!pip install -q langchain-aws>=0.1.0
!pip install -q langchain-community>=0.0.20
!pip install -q langgraph>=0.0.20
!pip install -q langsmith>=0.0.80
!pip install -q python-dotenv>=1.0.0
!pip install -q pydantic>=2.0.0

print("✅ All dependencies installed successfully!")
print("\n⚠️ If you see any warnings about kernel restart, please restart the kernel and re-run this cell.")

### Import Required Libraries

In [ ]:
# Standard library imports
import json
import os
import time
import uuid
from datetime import datetime
from typing import Dict, List, Optional, Any, TypedDict, Annotated
import warnings

# AWS SDK
import boto3
from botocore.exceptions import ClientError

# LangChain imports
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint.memory import MemorySaver

# LangSmith (optional - for tracing)
try:
    from langsmith import Client as LangSmithClient
    LANGSMITH_AVAILABLE = True
except ImportError:
    LANGSMITH_AVAILABLE = False
    print("⚠️ LangSmith not available - tracing will be disabled")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

### Configure AWS and LangSmith (Optional)

Set up your AWS credentials and optionally enable LangSmith tracing for observability.

In [ ]:
# AWS Configuration
AWS_REGION = os.environ.get('AWS_REGION', 'us-east-1')

# Initialize AWS clients
bedrock_client = boto3.client('bedrock', region_name=AWS_REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=AWS_REGION)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=AWS_REGION)
lambda_client = boto3.client('lambda', region_name=AWS_REGION)
iam_client = boto3.client('iam', region_name=AWS_REGION)
sts_client = boto3.client('sts', region_name=AWS_REGION)

# Get AWS account ID
AWS_ACCOUNT_ID = sts_client.get_caller_identity()['Account']

print(f"✅ AWS Configuration:")
print(f"   Region: {AWS_REGION}")
print(f"   Account ID: {AWS_ACCOUNT_ID}")

# LangSmith Configuration (Optional)
ENABLE_LANGSMITH = False  # Set to True if you have a LangSmith API key

if ENABLE_LANGSMITH and LANGSMITH_AVAILABLE:
    # Set these environment variables with your LangSmith credentials
    # os.environ["LANGCHAIN_TRACING_V2"] = "true"
    # os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-api-key"
    # os.environ["LANGCHAIN_PROJECT"] = "bedrock-agent-lab"
    print("\n✅ LangSmith tracing enabled")
else:
    print("\n⚠️ LangSmith tracing disabled (optional)")
    print("   To enable: Set ENABLE_LANGSMITH=True and configure LANGCHAIN_API_KEY")

### Verify Bedrock Model Access

Let's verify that you have access to the required Bedrock models.

In [ ]:
# Required models for this lab
REQUIRED_MODELS = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "amazon.titan-text-express-v1",
    "amazon.titan-embed-text-v2:0"
]

print("Checking Bedrock model access...\n")

try:
    # List foundation models
    response = bedrock_client.list_foundation_models()
    available_models = [model['modelId'] for model in response.get('modelSummaries', [])]
    
    all_available = True
    for model_id in REQUIRED_MODELS:
        if model_id in available_models:
            print(f"✅ {model_id}")
        else:
            print(f"❌ {model_id} - NOT AVAILABLE")
            all_available = False
    
    if all_available:
        print("\n✅ All required models are available!")
    else:
        print("\n⚠️ Some models are not available. Please enable them in the AWS Console:")
        print(f"   https://console.aws.amazon.com/bedrock/home?region={AWS_REGION}#/modelaccess")
        
except ClientError as e:
    print(f"❌ Error checking model access: {e}")
    print("\nPlease ensure you have proper IAM permissions for Bedrock.")

---

## Part 2: Helper Functions and Utilities

Let's define helper functions for IAM roles, Lambda functions, and resource management.

In [ ]:
# Global variables for resource tracking
CREATED_RESOURCES = {
    'iam_roles': [],
    'lambda_functions': [],
    'bedrock_agents': [],
    'action_groups': []
}

def create_iam_role_for_bedrock_agent(role_name: str) -> str:
    """
    Create an IAM role for Bedrock Agent with necessary permissions.
    
    Args:
        role_name: Name of the IAM role to create
        
    Returns:
        ARN of the created role
    """
    # Trust policy for Bedrock service
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": AWS_ACCOUNT_ID
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock:{AWS_REGION}:{AWS_ACCOUNT_ID}:agent/*"
                    }
                }
            }
        ]
    }
    
    # Permissions policy for Bedrock agent
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": [
                    f"arn:aws:bedrock:{AWS_REGION}::foundation-model/*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "lambda:InvokeFunction"
                ],
                "Resource": [
                    f"arn:aws:lambda:{AWS_REGION}:{AWS_ACCOUNT_ID}:function:*"
                ]
            }
        ]
    }
    
    try:
        # Try to get existing role
        response = iam_client.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']
        print(f"✅ Using existing IAM role: {role_name}")
        return role_arn
    except iam_client.exceptions.NoSuchEntityException:
        # Create new role
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description=f"Role for Bedrock Agent - {role_name}"
        )
        role_arn = response['Role']['Arn']
        
        # Attach inline policy
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f"{role_name}-policy",
            PolicyDocument=json.dumps(permissions_policy)
        )
        
        CREATED_RESOURCES['iam_roles'].append(role_name)
        print(f"✅ Created IAM role: {role_name}")
        print(f"   ARN: {role_arn}")
        
        # Wait for role to propagate
        print("   Waiting for IAM role to propagate (10 seconds)...")
        time.sleep(10)
        
        return role_arn

def create_lambda_execution_role(role_name: str) -> str:
    """
    Create an IAM role for Lambda function execution.
    
    Args:
        role_name: Name of the IAM role to create
        
    Returns:
        ARN of the created role
    """
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    try:
        response = iam_client.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']
        print(f"✅ Using existing Lambda execution role: {role_name}")
        return role_arn
    except iam_client.exceptions.NoSuchEntityException:
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description=f"Execution role for Lambda - {role_name}"
        )
        role_arn = response['Role']['Arn']
        
        # Attach AWS managed policy for basic Lambda execution
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
        
        CREATED_RESOURCES['iam_roles'].append(role_name)
        print(f"✅ Created Lambda execution role: {role_name}")
        print(f"   ARN: {role_arn}")
        
        # Wait for role to propagate
        print("   Waiting for IAM role to propagate (10 seconds)...")
        time.sleep(10)
        
        return role_arn

print("✅ Helper functions defined successfully!")

---

## Part 3: Create Lambda Functions for Action Groups

We'll create two Lambda functions:
1. **Weather Action Group** - Get weather information for cities
2. **Booking Action Group** - Create, search, and cancel bookings

### 3.1 Weather Lambda Function

In [ ]:
# Weather Lambda function code
WEATHER_LAMBDA_CODE = '''
import json
import random
from datetime import datetime

def lambda_handler(event, context):
    """
    Weather Action Group Lambda Handler.
    Simulates weather API calls for demonstration purposes.
    """
    print(f"Received event: {json.dumps(event)}")
    
    # Parse the action group request
    action_group = event.get('actionGroup', '')
    api_path = event.get('apiPath', '')
    parameters = event.get('parameters', [])
    
    # Convert parameters list to dict
    params_dict = {param['name']: param['value'] for param in parameters}
    
    # Handle different API paths
    if api_path == '/weather':
        return get_weather(params_dict)
    else:
        return {
            'statusCode': 404,
            'body': json.dumps({'error': f'Unknown API path: {api_path}'})
        }

def get_weather(params):
    """
    Get simulated weather data for a city.
    """
    city = params.get('city', 'Unknown')
    unit = params.get('unit', 'celsius').lower()
    
    # Simulate weather data
    conditions = ['sunny', 'cloudy', 'rainy', 'partly cloudy', 'clear']
    condition = random.choice(conditions)
    
    # Generate temperature based on unit
    if unit == 'fahrenheit':
        temp = random.randint(50, 85)
        temp_str = f"{temp}°F"
    else:
        temp = random.randint(10, 30)
        temp_str = f"{temp}°C"
    
    humidity = random.randint(40, 80)
    wind_speed = random.randint(5, 25)
    
    weather_data = {
        'city': city,
        'temperature': temp_str,
        'condition': condition,
        'humidity': f"{humidity}%",
        'wind_speed': f"{wind_speed} km/h",
        'timestamp': datetime.utcnow().isoformat()
    }
    
    # Format response for Bedrock Agent
    response = {
        'messageVersion': '1.0',
        'response': {
            'actionGroup': 'WeatherActionGroup',
            'apiPath': '/weather',
            'httpMethod': 'GET',
            'httpStatusCode': 200,
            'responseBody': {
                'application/json': {
                    'body': json.dumps(weather_data)
                }
            }
        }
    }
    
    return response
'''

def create_lambda_function(function_name: str, code: str, role_arn: str, description: str) -> Dict:
    """
    Create a Lambda function.
    
    Args:
        function_name: Name of the Lambda function
        code: Python code for the Lambda function
        role_arn: ARN of the execution role
        description: Description of the function
        
    Returns:
        Lambda function details
    """
    import zipfile
    from io import BytesIO
    
    try:
        # Try to get existing function
        response = lambda_client.get_function(FunctionName=function_name)
        print(f"✅ Using existing Lambda function: {function_name}")
        return response
    except lambda_client.exceptions.ResourceNotFoundException:
        # Create deployment package
        zip_buffer = BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.writestr('lambda_function.py', code)
        zip_buffer.seek(0)
        
        # Create Lambda function
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime='python3.11',
            Role=role_arn,
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': zip_buffer.read()},
            Description=description,
            Timeout=30,
            MemorySize=256,
            Publish=True
        )
        
        CREATED_RESOURCES['lambda_functions'].append(function_name)
        print(f"✅ Created Lambda function: {function_name}")
        print(f"   ARN: {response['FunctionArn']}")
        
        return response

# Create Lambda execution role
lambda_role_name = f"BedrockAgentLambdaRole-{uuid.uuid4().hex[:8]}"
lambda_role_arn = create_lambda_execution_role(lambda_role_name)

# Create Weather Lambda function
weather_function_name = f"WeatherActionGroup-{uuid.uuid4().hex[:8]}"
weather_lambda = create_lambda_function(
    function_name=weather_function_name,
    code=WEATHER_LAMBDA_CODE,
    role_arn=lambda_role_arn,
    description="Weather Action Group for Bedrock Agent"
)

WEATHER_LAMBDA_ARN = weather_lambda['FunctionArn']
print(f"\n✅ Weather Lambda ARN: {WEATHER_LAMBDA_ARN}")

### 3.2 Booking Lambda Function

In [ ]:
# Booking Lambda function code
BOOKING_LAMBDA_CODE = '''
import json
import uuid
from datetime import datetime

# In-memory booking storage (for demo purposes)
BOOKINGS = {}

def lambda_handler(event, context):
    """
    Booking Action Group Lambda Handler.
    Handles hotel/flight bookings with create, search, and cancel operations.
    """
    print(f"Received event: {json.dumps(event)}")
    
    # Parse the action group request
    api_path = event.get('apiPath', '')
    http_method = event.get('httpMethod', 'GET')
    parameters = event.get('parameters', [])
    
    # Convert parameters list to dict
    params_dict = {param['name']: param['value'] for param in parameters}
    
    # Route to appropriate handler
    if api_path == '/bookings' and http_method == 'POST':
        return create_booking(params_dict)
    elif api_path == '/bookings' and http_method == 'GET':
        return search_bookings(params_dict)
    elif api_path == '/bookings/cancel' and http_method == 'POST':
        return cancel_booking(params_dict)
    else:
        return error_response(404, f'Unknown API path: {api_path}')

def create_booking(params):
    """
    Create a new booking.
    """
    booking_type = params.get('type', 'hotel')
    customer_name = params.get('customer_name', 'Unknown')
    destination = params.get('destination', 'Unknown')
    check_in = params.get('check_in', '')
    check_out = params.get('check_out', '')
    price = float(params.get('price', 0))
    
    # Generate booking ID
    booking_id = str(uuid.uuid4())[:8]
    
    # Create booking record
    booking = {
        'booking_id': booking_id,
        'type': booking_type,
        'customer_name': customer_name,
        'destination': destination,
        'check_in': check_in,
        'check_out': check_out,
        'price': price,
        'status': 'confirmed',
        'created_at': datetime.utcnow().isoformat(),
        'requires_approval': price > 500  # Human approval for bookings > $500
    }
    
    BOOKINGS[booking_id] = booking
    
    return success_response(booking, 201)

def search_bookings(params):
    """
    Search for existing bookings.
    """
    customer_name = params.get('customer_name', '')
    
    # Filter bookings by customer name
    results = []
    for booking_id, booking in BOOKINGS.items():
        if customer_name.lower() in booking['customer_name'].lower():
            results.append(booking)
    
    return success_response({'bookings': results, 'count': len(results)})

def cancel_booking(params):
    """
    Cancel an existing booking.
    """
    booking_id = params.get('booking_id', '')
    
    if booking_id not in BOOKINGS:
        return error_response(404, f'Booking not found: {booking_id}')
    
    booking = BOOKINGS[booking_id]
    booking['status'] = 'cancelled'
    booking['cancelled_at'] = datetime.utcnow().isoformat()
    
    # Calculate refund (full refund for demo)
    refund_amount = booking['price']
    booking['refund_amount'] = refund_amount
    
    return success_response(booking)

def success_response(data, status_code=200):
    """Format successful response for Bedrock Agent."""
    return {
        'messageVersion': '1.0',
        'response': {
            'actionGroup': 'BookingActionGroup',
            'httpStatusCode': status_code,
            'responseBody': {
                'application/json': {
                    'body': json.dumps(data)
                }
            }
        }
    }

def error_response(status_code, message):
    """Format error response for Bedrock Agent."""
    return {
        'messageVersion': '1.0',
        'response': {
            'actionGroup': 'BookingActionGroup',
            'httpStatusCode': status_code,
            'responseBody': {
                'application/json': {
                    'body': json.dumps({'error': message})
                }
            }
        }
    }
'''

# Create Booking Lambda function
booking_function_name = f"BookingActionGroup-{uuid.uuid4().hex[:8]}"
booking_lambda = create_lambda_function(
    function_name=booking_function_name,
    code=BOOKING_LAMBDA_CODE,
    role_arn=lambda_role_arn,
    description="Booking Action Group for Bedrock Agent"
)

BOOKING_LAMBDA_ARN = booking_lambda['FunctionArn']
print(f"\n✅ Booking Lambda ARN: {BOOKING_LAMBDA_ARN}")

### 3.3 Grant Bedrock Permission to Invoke Lambda Functions

In [ ]:
def add_lambda_bedrock_permission(function_name: str):
    """
    Add resource-based policy to allow Bedrock to invoke Lambda function.
    """
    try:
        lambda_client.add_permission(
            FunctionName=function_name,
            StatementId='AllowBedrockInvoke',
            Action='lambda:InvokeFunction',
            Principal='bedrock.amazonaws.com',
            SourceAccount=AWS_ACCOUNT_ID,
            SourceArn=f'arn:aws:bedrock:{AWS_REGION}:{AWS_ACCOUNT_ID}:agent/*'
        )
        print(f"✅ Added Bedrock invoke permission to {function_name}")
    except lambda_client.exceptions.ResourceConflictException:
        print(f"✅ Bedrock invoke permission already exists for {function_name}")

# Add permissions for both Lambda functions
add_lambda_bedrock_permission(weather_function_name)
add_lambda_bedrock_permission(booking_function_name)

print("\n✅ Lambda functions configured successfully!")

---

## Part 4: Define OpenAPI Schemas for Action Groups

OpenAPI schemas define the capabilities and interfaces for our action groups.

In [ ]:
# Weather Action Group OpenAPI Schema
WEATHER_API_SCHEMA = {
    "openapi": "3.0.0",
    "info": {
        "title": "Weather API",
        "version": "1.0.0",
        "description": "API for retrieving weather information for cities worldwide"
    },
    "paths": {
        "/weather": {
            "get": {
                "summary": "Get current weather for a city",
                "description": "Returns current weather conditions including temperature, humidity, and wind speed",
                "operationId": "getWeather",
                "parameters": [
                    {
                        "name": "city",
                        "in": "query",
                        "description": "Name of the city",
                        "required": True,
                        "schema": {
                            "type": "string"
                        }
                    },
                    {
                        "name": "unit",
                        "in": "query",
                        "description": "Temperature unit (celsius or fahrenheit)",
                        "required": False,
                        "schema": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "default": "celsius"
                        }
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Successful response",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "city": {"type": "string"},
                                        "temperature": {"type": "string"},
                                        "condition": {"type": "string"},
                                        "humidity": {"type": "string"},
                                        "wind_speed": {"type": "string"},
                                        "timestamp": {"type": "string"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

# Booking Action Group OpenAPI Schema
BOOKING_API_SCHEMA = {
    "openapi": "3.0.0",
    "info": {
        "title": "Booking API",
        "version": "1.0.0",
        "description": "API for managing hotel and flight bookings"
    },
    "paths": {
        "/bookings": {
            "post": {
                "summary": "Create a new booking",
                "description": "Create a hotel or flight booking. Bookings over $500 require human approval.",
                "operationId": "createBooking",
                "parameters": [
                    {
                        "name": "type",
                        "in": "query",
                        "description": "Type of booking",
                        "required": True,
                        "schema": {
                            "type": "string",
                            "enum": ["hotel", "flight"]
                        }
                    },
                    {
                        "name": "customer_name",
                        "in": "query",
                        "description": "Name of the customer",
                        "required": True,
                        "schema": {"type": "string"}
                    },
                    {
                        "name": "destination",
                        "in": "query",
                        "description": "Destination city",
                        "required": True,
                        "schema": {"type": "string"}
                    },
                    {
                        "name": "check_in",
                        "in": "query",
                        "description": "Check-in date (YYYY-MM-DD)",
                        "required": True,
                        "schema": {"type": "string", "format": "date"}
                    },
                    {
                        "name": "check_out",
                        "in": "query",
                        "description": "Check-out date (YYYY-MM-DD)",
                        "required": True,
                        "schema": {"type": "string", "format": "date"}
                    },
                    {
                        "name": "price",
                        "in": "query",
                        "description": "Total price in USD",
                        "required": True,
                        "schema": {"type": "number"}
                    }
                ],
                "responses": {
                    "201": {
                        "description": "Booking created successfully"
                    }
                }
            },
            "get": {
                "summary": "Search bookings",
                "description": "Search for existing bookings by customer name",
                "operationId": "searchBookings",
                "parameters": [
                    {
                        "name": "customer_name",
                        "in": "query",
                        "description": "Customer name to search for",
                        "required": True,
                        "schema": {"type": "string"}
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Search results"
                    }
                }
            }
        },
        "/bookings/cancel": {
            "post": {
                "summary": "Cancel a booking",
                "description": "Cancel an existing booking and process refund",
                "operationId": "cancelBooking",
                "parameters": [
                    {
                        "name": "booking_id",
                        "in": "query",
                        "description": "Booking ID to cancel",
                        "required": True,
                        "schema": {"type": "string"}
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Booking cancelled successfully"
                    }
                }
            }
        }
    }
}

print("✅ OpenAPI schemas defined successfully!")
print(f"\nWeather API endpoints: {list(WEATHER_API_SCHEMA['paths'].keys())}")
print(f"Booking API endpoints: {list(BOOKING_API_SCHEMA['paths'].keys())}")

---

## Part 5: Create Bedrock Agents with Action Groups

Now we'll create two Bedrock Agents, each with their respective action groups.

### 5.1 Create Weather Agent

In [ ]:
# Create IAM role for Weather Agent
weather_agent_role_name = f"WeatherAgentRole-{uuid.uuid4().hex[:8]}"
weather_agent_role_arn = create_iam_role_for_bedrock_agent(weather_agent_role_name)

# Create Weather Agent
weather_agent_name = f"WeatherAgent-{uuid.uuid4().hex[:8]}"

print(f"Creating Weather Agent: {weather_agent_name}...")

weather_agent_response = bedrock_agent_client.create_agent(
    agentName=weather_agent_name,
    agentResourceRoleArn=weather_agent_role_arn,
    description="Agent for providing weather information",
    idleSessionTTLInSeconds=600,
    foundationModel="anthropic.claude-3-haiku-20240307-v1:0",
    instruction="""
You are a helpful weather assistant. Your role is to provide accurate and concise weather information for cities worldwide.

When users ask about weather:
1. Extract the city name from their query
2. Use the weather action group to retrieve current conditions
3. Present the information in a friendly, conversational manner
4. If the user doesn't specify temperature units, use Celsius by default

Always be polite and helpful. If you don't understand the city name, ask for clarification.
    """
)

weather_agent_id = weather_agent_response['agent']['agentId']
CREATED_RESOURCES['bedrock_agents'].append(weather_agent_id)

print(f"✅ Created Weather Agent: {weather_agent_id}")
print(f"   Name: {weather_agent_name}")
print(f"   Model: Claude 3 Haiku")

### 5.2 Create Action Group for Weather Agent

In [ ]:
# Create Weather Action Group
weather_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=weather_agent_id,
    agentVersion='DRAFT',
    actionGroupName='WeatherActionGroup',
    description='Get current weather information for cities',
    actionGroupExecutor={
        'lambda': WEATHER_LAMBDA_ARN
    },
    apiSchema={
        'payload': json.dumps(WEATHER_API_SCHEMA)
    },
    actionGroupState='ENABLED'
)

weather_action_group_id = weather_action_group_response['agentActionGroup']['actionGroupId']
print(f"✅ Created Weather Action Group: {weather_action_group_id}")

### 5.3 Create Booking Agent

In [ ]:
# Create IAM role for Booking Agent
booking_agent_role_name = f"BookingAgentRole-{uuid.uuid4().hex[:8]}"
booking_agent_role_arn = create_iam_role_for_bedrock_agent(booking_agent_role_name)

# Create Booking Agent
booking_agent_name = f"BookingAgent-{uuid.uuid4().hex[:8]}"

print(f"Creating Booking Agent: {booking_agent_name}...")

booking_agent_response = bedrock_agent_client.create_agent(
    agentName=booking_agent_name,
    agentResourceRoleArn=booking_agent_role_arn,
    description="Agent for managing hotel and flight bookings",
    idleSessionTTLInSeconds=600,
    foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
    instruction="""
You are a professional booking assistant for hotels and flights. Your role is to help customers create, search, and manage their bookings.

Key responsibilities:
1. **Creating Bookings**: Collect all required information (type, customer name, destination, dates, price) before creating
2. **High-Value Bookings**: For bookings over $500, inform the customer that human approval is required
3. **Searching Bookings**: Help customers find their existing bookings by name
4. **Cancellations**: Process booking cancellations and inform customers about refunds

Always:
- Be professional and courteous
- Confirm details before processing bookings
- Provide clear booking confirmation numbers
- Explain any approval requirements upfront

If information is missing, politely ask for it rather than making assumptions.
    """
)

booking_agent_id = booking_agent_response['agent']['agentId']
CREATED_RESOURCES['bedrock_agents'].append(booking_agent_id)

print(f"✅ Created Booking Agent: {booking_agent_id}")
print(f"   Name: {booking_agent_name}")
print(f"   Model: Claude 3 Sonnet")

### 5.4 Create Action Group for Booking Agent

In [ ]:
# Create Booking Action Group
booking_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=booking_agent_id,
    agentVersion='DRAFT',
    actionGroupName='BookingActionGroup',
    description='Manage hotel and flight bookings',
    actionGroupExecutor={
        'lambda': BOOKING_LAMBDA_ARN
    },
    apiSchema={
        'payload': json.dumps(BOOKING_API_SCHEMA)
    },
    actionGroupState='ENABLED'
)

booking_action_group_id = booking_action_group_response['agentActionGroup']['actionGroupId']
print(f"✅ Created Booking Action Group: {booking_action_group_id}")

### 5.5 Prepare Agents for Use

In [ ]:
def prepare_agent(agent_id: str, agent_name: str) -> str:
    """
    Prepare a Bedrock Agent for use.
    
    Args:
        agent_id: ID of the agent to prepare
        agent_name: Name of the agent (for logging)
        
    Returns:
        Agent alias ARN
    """
    print(f"\nPreparing agent: {agent_name}...")
    
    # Prepare the agent
    prepare_response = bedrock_agent_client.prepare_agent(agentId=agent_id)
    print(f"   Agent preparation initiated")
    
    # Wait for agent to be prepared
    max_attempts = 30
    for attempt in range(max_attempts):
        agent_status = bedrock_agent_client.get_agent(agentId=agent_id)
        status = agent_status['agent']['agentStatus']
        
        if status == 'PREPARED':
            print(f"   ✅ Agent prepared successfully")
            break
        elif status in ['FAILED', 'NOT_PREPARED']:
            raise Exception(f"Agent preparation failed with status: {status}")
        
        print(f"   Waiting for agent preparation... (attempt {attempt + 1}/{max_attempts})")
        time.sleep(10)
    else:
        raise Exception("Agent preparation timed out")
    
    # Create agent alias
    alias_name = f"{agent_name}-alias"
    alias_response = bedrock_agent_client.create_agent_alias(
        agentId=agent_id,
        agentAliasName=alias_name,
        description=f"Alias for {agent_name}"
    )
    
    alias_id = alias_response['agentAlias']['agentAliasId']
    print(f"   ✅ Created agent alias: {alias_id}")
    
    return alias_id

# Prepare both agents
weather_alias_id = prepare_agent(weather_agent_id, weather_agent_name)
booking_alias_id = prepare_agent(booking_agent_id, booking_agent_name)

print("\n" + "="*60)
print("✅ ALL AGENTS CREATED AND PREPARED SUCCESSFULLY!")
print("="*60)
print(f"\nWeather Agent ID: {weather_agent_id}")
print(f"Weather Alias ID: {weather_alias_id}")
print(f"\nBooking Agent ID: {booking_agent_id}")
print(f"Booking Alias ID: {booking_alias_id}")

---

## Part 6: Test Individual Agents (Single-Turn Interactions)

Let's test each agent independently before building the LangGraph orchestrator.

### 6.1 Test Weather Agent

In [ ]:
def invoke_bedrock_agent(agent_id: str, alias_id: str, session_id: str, prompt: str) -> Dict:
    """
    Invoke a Bedrock Agent and return the response.
    
    Args:
        agent_id: ID of the agent
        alias_id: ID of the agent alias
        session_id: Session ID for conversation tracking
        prompt: User prompt/query
        
    Returns:
        Agent response with completion text
    """
    response = bedrock_agent_runtime.invoke_agent(
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        inputText=prompt
    )
    
    # Extract completion from response stream
    completion = ""
    for event in response.get('completion', []):
        if 'chunk' in event:
            chunk = event['chunk']
            if 'bytes' in chunk:
                completion += chunk['bytes'].decode('utf-8')
    
    return {
        'completion': completion,
        'session_id': session_id
    }

# Test Weather Agent
print("Testing Weather Agent...\n")

weather_session_id = str(uuid.uuid4())
weather_test_queries = [
    "What's the weather in London?",
    "How about Tokyo in Fahrenheit?",
    "Tell me about the weather in Paris"
]

for query in weather_test_queries:
    print(f"User: {query}")
    response = invoke_bedrock_agent(
        agent_id=weather_agent_id,
        alias_id=weather_alias_id,
        session_id=weather_session_id,
        prompt=query
    )
    print(f"Agent: {response['completion']}")
    print("-" * 60)
    time.sleep(2)  # Rate limiting

### 6.2 Test Booking Agent

In [ ]:
# Test Booking Agent
print("Testing Booking Agent...\n")

booking_session_id = str(uuid.uuid4())
booking_test_queries = [
    "Create a hotel booking for John Smith in New York from March 15 to March 20, 2026 for $350",
    "Search for bookings under John Smith",
    "Create a flight booking for Jane Doe to Tokyo from April 1 to April 10, 2026 for $850"
]

for query in booking_test_queries:
    print(f"User: {query}")
    response = invoke_bedrock_agent(
        agent_id=booking_agent_id,
        alias_id=booking_alias_id,
        session_id=booking_session_id,
        prompt=query
    )
    print(f"Agent: {response['completion']}")
    print("-" * 60)
    time.sleep(2)  # Rate limiting

---

## Part 7: Build LangGraph Orchestrator for Multi-Agent Workflows

Now let's build an advanced LangGraph orchestrator that can:
- Route queries to the appropriate agent
- Maintain conversation state
- Handle multi-turn conversations
- Implement human-in-the-loop approval for high-value bookings

### 7.1 Define State Schema

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    """
    State schema for the multi-agent orchestrator.
    """
    # Conversation history
    messages: Annotated[Sequence[BaseMessage], operator.add]
    
    # Current user input
    current_input: str
    
    # Routing decision
    next_agent: str
    
    # Agent responses
    weather_response: str
    booking_response: str
    
    # Session tracking
    session_id: str
    
    # Human approval tracking
    requires_approval: bool
    approval_message: str
    
    # Final output
    final_response: str

print("✅ State schema defined")

### 7.2 Create Router Node

In [ ]:
# Initialize Claude for routing decisions
router_llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    region_name=AWS_REGION,
    client=bedrock_runtime
)

def route_query(state: AgentState) -> AgentState:
    """
    Route user query to the appropriate agent based on content.
    """
    user_input = state['current_input']
    
    # Use Claude to classify the query
    routing_prompt = f"""Analyze this user query and determine which agent should handle it.

User Query: {user_input}

Available Agents:
- weather: For weather-related queries
- booking: For hotel/flight booking queries
- general: For general conversation or unclear queries

Respond with ONLY the agent name (weather, booking, or general)."""
    
    messages = [HumanMessage(content=routing_prompt)]
    response = router_llm.invoke(messages)
    
    agent_choice = response.content.strip().lower()
    
    # Validate agent choice
    if agent_choice not in ['weather', 'booking', 'general']:
        agent_choice = 'general'
    
    state['next_agent'] = agent_choice
    print(f"🔀 Routing to: {agent_choice}")
    
    return state

print("✅ Router node created")

### 7.3 Create Agent Nodes

In [ ]:
def weather_agent_node(state: AgentState) -> AgentState:
    """
    Invoke weather agent and store response.
    """
    print("🌤️  Invoking Weather Agent...")
    
    response = invoke_bedrock_agent(
        agent_id=weather_agent_id,
        alias_id=weather_alias_id,
        session_id=state.get('session_id', str(uuid.uuid4())),
        prompt=state['current_input']
    )
    
    state['weather_response'] = response['completion']
    state['final_response'] = response['completion']
    state['messages'].append(AIMessage(content=response['completion']))
    
    return state

def booking_agent_node(state: AgentState) -> AgentState:
    """
    Invoke booking agent and check for approval requirements.
    """
    print("📅 Invoking Booking Agent...")
    
    response = invoke_bedrock_agent(
        agent_id=booking_agent_id,
        alias_id=booking_alias_id,
        session_id=state.get('session_id', str(uuid.uuid4())),
        prompt=state['current_input']
    )
    
    completion = response['completion']
    state['booking_response'] = completion
    
    # Check if approval is needed (bookings over $500)
    if '$' in state['current_input']:
        # Extract price from input
        import re
        price_match = re.search(r'\$([0-9,]+)', state['current_input'])
        if price_match:
            price = float(price_match.group(1).replace(',', ''))
            if price > 500:
                state['requires_approval'] = True
                state['approval_message'] = f"\n\n⚠️ HUMAN APPROVAL REQUIRED: This booking exceeds $500 (${price:.2f})"
                completion += state['approval_message']
    
    state['final_response'] = completion
    state['messages'].append(AIMessage(content=completion))
    
    return state

def general_response_node(state: AgentState) -> AgentState:
    """
    Handle general queries that don't need specialized agents.
    """
    print("💬 Handling general query...")
    
    # Use Claude for general responses
    general_llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        region_name=AWS_REGION,
        client=bedrock_runtime
    )
    
    messages = [HumanMessage(content=state['current_input'])]
    response = general_llm.invoke(messages)
    
    state['final_response'] = response.content
    state['messages'].append(AIMessage(content=response.content))
    
    return state

print("✅ Agent nodes created")

### 7.4 Build LangGraph Workflow

In [ ]:
from langgraph.graph import StateGraph, END

def should_continue(state: AgentState) -> str:
    """
    Determine next node based on routing decision.
    """
    next_agent = state.get('next_agent', 'general')
    
    if next_agent == 'weather':
        return 'weather'
    elif next_agent == 'booking':
        return 'booking'
    else:
        return 'general'

# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("router", route_query)
workflow.add_node("weather", weather_agent_node)
workflow.add_node("booking", booking_agent_node)
workflow.add_node("general", general_response_node)

# Add edges
workflow.set_entry_point("router")
workflow.add_conditional_edges(
    "router",
    should_continue,
    {
        "weather": "weather",
        "booking": "booking",
        "general": "general"
    }
)
workflow.add_edge("weather", END)
workflow.add_edge("booking", END)
workflow.add_edge("general", END)

# Compile graph
app = workflow.compile()

print("✅ LangGraph workflow compiled successfully!")
print("\nWorkflow structure:")
print("  1. Router (classify query)")
print("  2. Agent nodes (weather/booking/general)")
print("  3. Response generation")

---

## Part 8: Run Multi-Turn Conversations

Let's test the complete orchestrator with complex multi-turn scenarios.

In [ ]:
def run_conversation(queries: List[str], session_id: str = None) -> str:
    """
    Run a multi-turn conversation through the LangGraph orchestrator.
    
    Args:
        queries: List of user queries
        session_id: Optional session ID for tracking
        
    Returns:
        Session ID for continued conversation
    """
    if session_id is None:
        session_id = str(uuid.uuid4())
    
    print("="*60)
    print(f"Starting Conversation (Session: {session_id[:8]}...)")
    print("="*60)
    
    for i, query in enumerate(queries, 1):
        print(f"\n👤 Turn {i}: {query}")
        print("-" * 60)
        
        # Prepare state
        state = {
            'messages': [],
            'current_input': query,
            'next_agent': '',
            'weather_response': '',
            'booking_response': '',
            'session_id': session_id,
            'requires_approval': False,
            'approval_message': '',
            'final_response': ''
        }
        
        # Run workflow
        result = app.invoke(state)
        
        # Display response
        print(f"\n🤖 Agent: {result['final_response']}")
        print("-" * 60)
        
        # Small delay between turns
        if i < len(queries):
            time.sleep(2)
    
    print("\n" + "="*60)
    print("Conversation Complete")
    print("="*60)
    
    return session_id

print("✅ Conversation runner ready")

### 8.1 Scenario 1: Weather Queries

In [ ]:
weather_conversation = [
    "What's the weather like in London today?",
    "How about in Tokyo?",
    "And what's the temperature in New York in Fahrenheit?"
]

weather_session = run_conversation(weather_conversation)

### 8.2 Scenario 2: Booking Flow with Human Approval

In [ ]:
booking_conversation = [
    "I need to book a hotel in Paris",
    "It's for Jane Smith, March 20-25, 2026, and the price is $450",
    "Can you search for bookings under Jane Smith?",
    "Actually, I also need a flight to Tokyo for John Doe, April 1-10, 2026, for $850"
]

booking_session = run_conversation(booking_conversation)

### 8.3 Scenario 3: Mixed Queries (Weather + Booking)

In [ ]:
mixed_conversation = [
    "What's the weather in Barcelona?",
    "Great! I'd like to book a hotel there for Sarah Williams",
    "The dates are May 5-12, 2026, and it costs $600",
    "What will the weather be like in Rome?",
    "Can you find all bookings for Sarah Williams?"
]

mixed_session = run_conversation(mixed_conversation)

---

## Part 9: Advanced Features - Error Handling and Observability

Let's add production-grade error handling and observability.

### 9.1 Enhanced Error Handling

In [ ]:
import traceback
from functools import wraps

def with_retry(max_retries: int = 3, backoff: float = 2.0):
    """
    Decorator for retry logic with exponential backoff.
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        print(f"❌ All retry attempts failed for {func.__name__}")
                        raise
                    
                    wait_time = backoff ** attempt
                    print(f"⚠️ Attempt {attempt + 1} failed: {str(e)}")
                    print(f"   Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
        return wrapper
    return decorator

@with_retry(max_retries=3)
def invoke_agent_with_retry(agent_id: str, alias_id: str, session_id: str, prompt: str) -> Dict:
    """
    Invoke agent with automatic retry logic.
    """
    return invoke_bedrock_agent(agent_id, alias_id, session_id, prompt)

print("✅ Error handling configured")

### 9.2 LangSmith Integration (Optional)

In [ ]:
if ENABLE_LANGSMITH and LANGSMITH_AVAILABLE:
    print("Setting up LangSmith tracing...")
    
    # LangSmith is automatically configured through environment variables
    # You can view traces at: https://smith.langchain.com/
    
    print("✅ LangSmith tracing active")
    print(f"   Project: {os.environ.get('LANGCHAIN_PROJECT', 'default')}")
    print("   View traces at: https://smith.langchain.com/")
else:
    print("⚠️ LangSmith tracing not enabled")
    print("   To enable: Set ENABLE_LANGSMITH=True and configure LANGCHAIN_API_KEY")

---

## Part 10: Production Deployment Checklist

Before deploying to production, review these critical items.

In [ ]:
production_checklist = {
    "Infrastructure": [
        "✓ IAM roles follow least-privilege principle",
        "✓ Lambda functions have appropriate timeout and memory",
        "✓ CloudWatch logs configured for all components",
        "✓ Resource tagging for cost tracking"
    ],
    "Security": [
        "✓ Secrets stored in AWS Secrets Manager",
        "✓ VPC endpoints for private Bedrock access",
        "✓ Encryption at rest and in transit",
        "✓ Security group and NACL rules configured"
    ],
    "Monitoring": [
        "✓ CloudWatch alarms for errors and latency",
        "✓ X-Ray tracing enabled",
        "✓ LangSmith observability configured",
        "✓ Cost monitoring and alerts"
    ],
    "Reliability": [
        "✓ Retry logic with exponential backoff",
        "✓ Circuit breaker pattern for failures",
        "✓ Graceful degradation strategies",
        "✓ Rate limiting and throttling"
    ],
    "Testing": [
        "✓ Unit tests for Lambda functions",
        "✓ Integration tests for agent workflows",
        "✓ Load testing for performance",
        "✓ Security testing and penetration tests"
    ]
}

print("📋 Production Deployment Checklist")
print("="*60)
for category, items in production_checklist.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  {item}")

---

## Part 11: Cleanup Resources

**IMPORTANT:** Run this section to delete all created resources and avoid ongoing charges.

In [ ]:
def cleanup_lab_resources(confirm: bool = False):
    """
    Delete all resources created during the lab.
    
    Args:
        confirm: Set to True to actually delete resources
    """
    if not confirm:
        print("⚠️ Cleanup not confirmed. Set confirm=True to delete resources.")
        return
    
    print("🧹 Starting resource cleanup...")
    print("="*60)
    
    # Delete Bedrock Agents
    for agent_id in CREATED_RESOURCES['bedrock_agents']:
        try:
            bedrock_agent_client.delete_agent(agentId=agent_id, skipResourceInUseCheck=True)
            print(f"✅ Deleted Bedrock Agent: {agent_id}")
        except Exception as e:
            print(f"❌ Failed to delete agent {agent_id}: {e}")
    
    # Delete Lambda Functions
    for function_name in CREATED_RESOURCES['lambda_functions']:
        try:
            lambda_client.delete_function(FunctionName=function_name)
            print(f"✅ Deleted Lambda function: {function_name}")
        except Exception as e:
            print(f"❌ Failed to delete function {function_name}: {e}")
    
    # Delete IAM Roles
    for role_name in CREATED_RESOURCES['iam_roles']:
        try:
            # Detach managed policies
            try:
                iam_client.detach_role_policy(
                    RoleName=role_name,
                    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
                )
            except:
                pass
            
            # Delete inline policies
            policies = iam_client.list_role_policies(RoleName=role_name)
            for policy_name in policies.get('PolicyNames', []):
                iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
            
            # Delete role
            iam_client.delete_role(RoleName=role_name)
            print(f"✅ Deleted IAM role: {role_name}")
        except Exception as e:
            print(f"❌ Failed to delete role {role_name}: {e}")
    
    print("\n" + "="*60)
    print("✅ Cleanup complete!")
    print("="*60)

# To delete resources, uncomment the line below and run:
# cleanup_lab_resources(confirm=True)

print("⚠️ To cleanup resources, run: cleanup_lab_resources(confirm=True)")

---

## Summary and Next Steps

### 🎉 Congratulations!

You've successfully completed this comprehensive lab on building production-grade AI agents with AWS Bedrock, LangGraph, and LangSmith!

### What You've Learned

1. ✅ Created AWS Bedrock Agents with custom action groups
2. ✅ Designed OpenAPI schemas for agent capabilities
3. ✅ Implemented Lambda functions as action group backends
4. ✅ Built multi-agent workflows with LangGraph
5. ✅ Implemented stateful conversations with memory
6. ✅ Added human-in-the-loop approval patterns
7. ✅ Handled errors gracefully with retry logic
8. ✅ Integrated LangSmith for observability

### Next Steps

1. **Experiment**: Modify agent instructions and test different scenarios
2. **Extend**: Add more action groups and capabilities
3. **Optimize**: Tune model selection and prompts for performance
4. **Deploy**: Use the production checklist to deploy safely
5. **Monitor**: Set up comprehensive observability with LangSmith

### Additional Resources

- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [LangChain Documentation](https://python.langchain.com/)
- [LangGraph Guides](https://python.langchain.com/docs/langgraph)
- [LangSmith Platform](https://smith.langchain.com/)

### Cost Management

**Don't forget to run the cleanup cell above to delete resources and avoid ongoing charges!**

---

**Questions or Issues?**

- Review the troubleshooting section in the README
- Check CloudWatch logs for detailed error messages
- Verify IAM permissions and model access

Thank you for completing this lab! 🚀